In [9]:
#allow autoreload modules at every execution if requested
%load_ext autoreload

import tempfile
import unittest

from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd

from cs_parser import CsParser, mappings
from star_parser import StarTabDf, StarParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload 2
work_folder = Path("/mnt/home/a.murachelli/Downloads/")
micrographs_file = work_folder/"imported_movies.cs"
micrographs_file.exists()


True

In [11]:
# link_folder = Path("/mnt/DATA/andrea/AM_20211112_NeCen_BRCA1AUb/cryolo_train/micrographs")
# project_prefix = '/mnt/DATA/andrea/AM_20211112_NeCen_BRCA1AUb/P112/' #to be added to the link target because cryosparc only adds job number to the path;
# update_links = True #overwrite links if they already exist
# try:
#     assert micrographs_file.exists()
# except AssertionError:
#     raise AssertionError(f'File {micrographs_file} does not exist')
# particles_file = work_folder/"J89_particles_accepted.cs"
# particle_starfile = work_folder/"J89_particles_selected.star"
parser = CsParser(micrographs_file)
array = parser.parse_array()

In [12]:

##process micrograph blob
# remove multidimensional arrays
a = array.cs_array.copy()
dtypes = a.dtype.fields
problematic = []
clean = []
for index, value in enumerate(list(a[0])):
    if isinstance(value, np.ndarray):
        key = list(a.dtype.fields.keys())[index]
        problematic.append(key)
    else:
        key = list(a.dtype.fields.keys())[index]
        clean.append(key)
good = a[clean]
#process micrograph paths 
df = pd.DataFrame(good)


In [18]:
import shutil
df1 = list(df['movie_blob/path'].apply(lambda x: x.decode('utf-8')))
df2 = [x[len('J5/imported/016669088211364111591_'):] for x in df1]
destination = Path('/em-images/2022/NK_20220301_NeCen/Micrographs/50_frames')
origin = Path('/em-images/2022/NK_20220301_NeCen/Micrographs')
for mic in df2:
    dest_m = destination/mic
    origin_m = origin/mic
    assert origin_m.exists()
    shutil.move(origin_m, dest_m)


In [24]:
for mic in df2:
    xml = mic[:-4] + 'xml'
    dest_xml = destination/xml
    origin_xml = origin/xml
    shutil.move(origin_xml, dest_xml)

In [ ]:
mics = df['movie_blob/path'].apply(lambda x: project_prefix + x.decode('utf-8'))
mics = [work_folder/i for i in mics]
#make links
if not link_folder.exists():
    link_folder.mkdir(parents=True)
for m in mics:
    print(f'mic is {m}')
    assert m.exists()
    link_name = link_folder/m.name
    if update_links:
        link_name.unlink(missing_ok=True)
    print(f'link is {link_name}')
    break
    # link_name.symlink_to(m, )
